## Kaggle score: 

- with fe data v0: MAE=18243
- with fe data v1: MAE=18243 (no change)

# Libraries

In [15]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score,mean_squared_error,mean_absolute_error,auc,roc_curve,confusion_matrix
from xgboost import XGBRegressor

In [16]:
train = pd.read_csv('/home/giampa95/programming/DS/housing_price_kaggle/data/train_fe_v1.csv')
test = pd.read_csv('/home/giampa95/programming/DS/housing_price_kaggle/data/test_fe_v1.csv')

# Fit model

# Define predictors and target

In [17]:
# Converto to numpy
x_columns = train.columns.drop(['SalePrice','Unnamed: 0' ])
x = train[x_columns].values
y = train['SalePrice'].values
x_columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       ...
       'WoodDeckSF0.5', 'WoodDeckSF2', 'PoolQC0.5', 'PoolQC2', 'RoofMatl0.5',
       'RoofMatl2', 'HalfBath0.5', 'HalfBath2', 'LandContour0.5',
       'LandContour2'],
      dtype='object', length=137)

## Split data

In [18]:
# Create train/test
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.4, random_state=5)

# Model

In [29]:
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
model.fit(x_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(x_train, y_train)], 
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [30]:
y_pred = model.predict(x_test)
R2 = r2_score(y_test,y_pred)
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)

In [31]:
print(f'R2 = {round(R2,3)} and MAE = {round(MAE,3)}')

R2 = 0.922 and MAE = 14813.064


# Making predictions

In [32]:
test.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,WoodDeckSF0.5,WoodDeckSF2,PoolQC0.5,PoolQC2,RoofMatl0.5,RoofMatl2,HalfBath0.5,HalfBath2,LandContour0.5,LandContour2
0,0,1461,0.117647,0.50,0.255591,0.054322,1.0,1.0,0.000000,0.333333,...,0.404179,0.026687,0.0,0.0,0.534522,0.081633,0.000000,0.00,0.57735,0.111111
1,1,1462,0.117647,0.75,0.258786,0.066685,1.0,1.0,0.333333,0.333333,...,0.677183,0.210292,0.0,0.0,0.534522,0.081633,0.707107,0.25,0.57735,0.111111
2,2,1463,0.352941,0.75,0.236422,0.064643,1.0,1.0,0.333333,0.333333,...,0.497368,0.061194,0.0,0.0,0.534522,0.081633,0.707107,0.25,0.57735,0.111111
3,3,1464,0.352941,0.75,0.249201,0.046638,1.0,1.0,0.333333,0.333333,...,0.648128,0.176459,0.0,0.0,0.534522,0.081633,0.707107,0.25,0.57735,0.111111
4,4,1465,0.705882,0.75,0.137380,0.023394,1.0,1.0,0.333333,1.000000,...,0.000000,0.000000,0.0,0.0,0.534522,0.081633,0.000000,0.00,1.00000,1.000000


In [33]:
x_ids = test['Id']
test_columns = test.columns.drop(['Unnamed: 0','Id'])
x_test_2 = test[test_columns].values

x_ids


0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [34]:
y_pred_2 = model.predict(x_test_2)

final=pd.DataFrame(data={'Id':x_ids,'SalePrice':y_pred_2})
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id         1459 non-null   int64  
 1   SalePrice  1459 non-null   float32
dtypes: float32(1), int64(1)
memory usage: 17.2 KB


# Submiting 

In [35]:
# Run the code to save predictions in the format used for competition scoring

final.to_csv('../submissions/submission_xgboost_v1.csv',index=False)